In [31]:
'''
Rough Draft file for GNN testing. Using for testing different ideas, methods, approaches, etc.

-- Some citations present here that could be useful, review if needed
'''

#Install the required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler


import torch
from torch_geometric.datasets import KarateClub
from torch_geometric.utils import to_networkx
from torch_geometric.data import Data
import networkx as nx  
import matplotlib.pyplot as plt
import numpy as np
from torch.nn import Linear
from torch_geometric.nn import GCNConv  

In [32]:
df = pd.read_csv("Data/features_3_sec.csv")
df.head()


filename  length  chroma_stft_mean  chroma_stft_var  rms_mean  \
0  blues.00000.0.wav   66149          0.335406         0.091048  0.130405   
1  blues.00000.1.wav   66149          0.343065         0.086147  0.112699   
2  blues.00000.2.wav   66149          0.346815         0.092243  0.132003   
3  blues.00000.3.wav   66149          0.363639         0.086856  0.132565   
4  blues.00000.4.wav   66149          0.335579         0.088129  0.143289   

    rms_var  spectral_centroid_mean  spectral_centroid_var  \
0  0.003521             1773.065032          167541.630869   
1  0.001450             1816.693777           90525.690866   
2  0.004620             1788.539719          111407.437613   
3  0.002448             1655.289045          111952.284517   
4  0.001701             1630.656199           79667.267654   

   spectral_bandwidth_mean  spectral_bandwidth_var  ...  mfcc16_var  \
0              1972.744388           117335.771563  ...   39.687145   
1              2010.051501            65671.875673  ...   64.748276   
2              2084.565132            75124.921716  ...   67.336563   
3              1960.039988            82913.639269  ...   47.739452   
4              1948.503884            60204.020268  ...   30.336359   

   mfcc17_mean  mfcc17_var  mfcc18_mean  mfcc18_var  mfcc19_mean  mfcc19_var  \
0    -3.241280   36.488243     0.722209   38.099152    -5.050335   33.618073   
1    -6.055294   40.677654     0.159015   51.264091    -2.837699   97.030830   
2    -1.768610   28.348579     2.378768   45.717648    -1.938424   53.050835   
3    -3.841155   28.337118     1.218588   34.770935    -3.580352   50.836224   
4     0.664582   45.880913     1.689446   51.363583    -3.392489   26.738789   

   mfcc20_mean  mfcc20_var  label  
0    -0.243027   43.771767  blues  
1     5.784063   59.943081  blues  
2     2.517375   33.105122  blues  
3     3.630866   32.023678  blues  
4     0.536961   29.146694  blues  

[5 rows x 60 columns]

In [33]:
# Label Encoding - encod the categorical classes with numerical integer values for training

# Blues - 0
# Classical - 1
# Country - 2
# Disco - 3
# Hip-hop - 4 
# Jazz - 5  
# Metal - 6 
# Pop - 7
# Reggae - 8
# Rock - 9

#To convert categorical data into model-understandable numerical data
class_list = df.iloc[:, -1]
convertor = LabelEncoder()

df = df.drop(labels="filename", axis=1)
df = df.drop(labels="length", axis=1)
df.head()

chroma_stft_mean  chroma_stft_var  rms_mean   rms_var  \
0          0.335406         0.091048  0.130405  0.003521   
1          0.343065         0.086147  0.112699  0.001450   
2          0.346815         0.092243  0.132003  0.004620   
3          0.363639         0.086856  0.132565  0.002448   
4          0.335579         0.088129  0.143289  0.001701   

   spectral_centroid_mean  spectral_centroid_var  spectral_bandwidth_mean  \
0             1773.065032          167541.630869              1972.744388   
1             1816.693777           90525.690866              2010.051501   
2             1788.539719          111407.437613              2084.565132   
3             1655.289045          111952.284517              1960.039988   
4             1630.656199           79667.267654              1948.503884   

   spectral_bandwidth_var  rolloff_mean   rolloff_var  ...  mfcc16_var  \
0           117335.771563   3714.560359  1.080790e+06  ...   39.687145   
1            65671.875673   3869.682242  6.722448e+05  ...   64.748276   
2            75124.921716   3997.639160  7.907127e+05  ...   67.336563   
3            82913.639269   3568.300218  9.216524e+05  ...   47.739452   
4            60204.020268   3469.992864  6.102111e+05  ...   30.336359   

   mfcc17_mean  mfcc17_var  mfcc18_mean  mfcc18_var  mfcc19_mean  mfcc19_var  \
0    -3.241280   36.488243     0.722209   38.099152    -5.050335   33.618073   
1    -6.055294   40.677654     0.159015   51.264091    -2.837699   97.030830   
2    -1.768610   28.348579     2.378768   45.717648    -1.938424   53.050835   
3    -3.841155   28.337118     1.218588   34.770935    -3.580352   50.836224   
4     0.664582   45.880913     1.689446   51.363583    -3.392489   26.738789   

   mfcc20_mean  mfcc20_var  label  
0    -0.243027   43.771767  blues  
1     5.784063   59.943081  blues  
2     2.517375   33.105122  blues  
3     3.630866   32.023678  blues  
4     0.536961   29.146694  blues  

[5 rows x 58 columns]

In [34]:
#Fitting the label encoder & return encoded labels
y = convertor.fit_transform(class_list)
y

array([0, 0, 0, ..., 9, 9, 9])

In [35]:
df = df.drop(labels="label", axis=1)
df.head()

chroma_stft_mean  chroma_stft_var  rms_mean   rms_var  \
0          0.335406         0.091048  0.130405  0.003521   
1          0.343065         0.086147  0.112699  0.001450   
2          0.346815         0.092243  0.132003  0.004620   
3          0.363639         0.086856  0.132565  0.002448   
4          0.335579         0.088129  0.143289  0.001701   

   spectral_centroid_mean  spectral_centroid_var  spectral_bandwidth_mean  \
0             1773.065032          167541.630869              1972.744388   
1             1816.693777           90525.690866              2010.051501   
2             1788.539719          111407.437613              2084.565132   
3             1655.289045          111952.284517              1960.039988   
4             1630.656199           79667.267654              1948.503884   

   spectral_bandwidth_var  rolloff_mean   rolloff_var  ...  mfcc16_mean  \
0           117335.771563   3714.560359  1.080790e+06  ...    -2.853603   
1            65671.875673   3869.682242  6.722448e+05  ...     4.074709   
2            75124.921716   3997.639160  7.907127e+05  ...     4.806280   
3            82913.639269   3568.300218  9.216524e+05  ...    -1.359111   
4            60204.020268   3469.992864  6.102111e+05  ...     2.092937   

   mfcc16_var  mfcc17_mean  mfcc17_var  mfcc18_mean  mfcc18_var  mfcc19_mean  \
0   39.687145    -3.241280   36.488243     0.722209   38.099152    -5.050335   
1   64.748276    -6.055294   40.677654     0.159015   51.264091    -2.837699   
2   67.336563    -1.768610   28.348579     2.378768   45.717648    -1.938424   
3   47.739452    -3.841155   28.337118     1.218588   34.770935    -3.580352   
4   30.336359     0.664582   45.880913     1.689446   51.363583    -3.392489   

   mfcc19_var  mfcc20_mean  mfcc20_var  
0   33.618073    -0.243027   43.771767  
1   97.030830     5.784063   59.943081  
2   53.050835     2.517375   33.105122  
3   50.836224     3.630866   32.023678  
4   26.738789     0.536961   29.146694  

[5 rows x 57 columns]

In [36]:
# x_train,x_test,y_train,y_test=train_test_split(df.iloc[:,:-1],y,test_size=0.3)
# x_train.head()
# # y_train.item(1)
# x_train.shape

In [37]:
# df = df.drop(labels="label", axis=1)

In [38]:
'''
The following is done to try and determine the correlation between all features. 
This will help to know which features should have edges connecting them and which do not need them,
allowing the graph to have less edges and optimize the GNN.

Some logic is derived from the following Stack Overflow:
https://stackoverflow.com/questions/29432629/plot-correlation-matrix-using-pandas

Correlation ideas:
https://arxiv.org/pdf/2206.07743
'''

corr = df.corr()
corr.style.background_gradient(cmap='coolwarm')

In [39]:
correlation_matrix = df.corr()
threshold = 0.2

G = nx.Graph()

# Add nodes (features)
features = df.columns
G.add_nodes_from(features)
G.number_of_nodes()

# plt.figure(figsize=(12, 8))
# pos = nx.spring_layout(G)
# nx.draw(G, pos, with_labels=True, node_color="lightblue", edge_color="gray")
# plt.title("Correlation-Based Graph")
# plt.show()
# correlation_matrix


57

In [54]:
# from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
from numpy.linalg import norm

scaler = StandardScaler()
X_scaled = scaler.fit_transform(df)
print(X_scaled)

col1 = np.array(df['chroma_stft_mean'])
col2 = np.array(df['mfcc17_var'])

cosine = np.dot(col1, col2)/(norm(col1)*norm(col2))
print("cosine similarity: ", cosine)

similarity_matrix = cosine_similarity(X_scaled)

k = 10  # Number of neighbors
edges = []
count = 0
for i in range(len(X_scaled)):
    neighbors = np.argsort(-similarity_matrix[i])[:k+1]  # Sort descending
    for neighbor in neighbors:
        if neighbor != i:  # Exclude self-loops
            edges.append([i, neighbor])
            edges.append([neighbor, i])
            count = count + 1

# Convert edges to PyTorch tensor
edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()

print(f"Number of nodes: {len(X_scaled)}")
print(f"Number of edges: {edge_index.shape[1]}")
print(f"num edges: {count}")

# Add edges based on k-nearest neighbors
# edges = []
# for i in range(len(features)):
#     distances = np.argsort(similarity_matrix[i])
#     nearest_neighbors = distances[1:k+1]  # Exclude self (index 0)
#     for neighbor in nearest_neighbors:
#         G.add_edge(features[i], features[neighbor], weight=distance_matrix[i, neighbor])
#         edges.append([i, j])
#         edges.append([j, i])


# edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()

# similarity_matrix
edge_index


[[-0.48780784  0.64052047 -0.00662408 ... -0.51356204  0.12841417
  -0.29178072]
 [-0.40314187  0.13183473 -0.26494432 ...  1.01138445  1.27578001
   0.05642464]
 [-0.36169428  0.7644909   0.01669533 ... -0.04624405  0.65390663
  -0.52145798]
 ...
 [-0.35433044  0.42997426 -1.14464442 ... -0.15370124  0.11765485
  -0.33882395]
 [ 0.0883611  -0.00630133 -0.93999575 ... -0.72456977  0.30333409
  -0.95893743]
 [-0.11321002  0.19536324 -1.17205474 ... -0.37245283 -0.47495901
  -0.55112155]]
cosine similarity:  0.760302485713915
Number of nodes: 9990
Number of edges: 199800
num edges: 99900


tensor([[   0,    7,    0,  ..., 9984, 9989, 9944],
        [   7,    0,    3,  ..., 9989, 9944, 9989]])

In [41]:
import torch.nn.functional as F

class GCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCN, self).__init__()
        # self.conv1 = GCNConv(input_dim, hidden_dim)
        # self.conv2 = GCNConv(hidden_dim, output_dim)
        # self.relu = torch.nn.ReLU()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.bn1 = torch.nn.BatchNorm1d(hidden_dim)  # Batch Norm after the first layer
        self.conv2 = GCNConv(hidden_dim, output_dim)
        self.dropout = torch.nn.Dropout(0.5)  # Dropout for regularization


    def forward(self, x, edge_index):
        # x = self.conv1(x, edge_index)
        # x = self.relu(x)
        # x = self.conv2(x, edge_index)
        # return x
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.bn1(x)  # Batch Normalization
        x = self.dropout(x)  # Dropout
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)  # Log-Softmax for classification


In [52]:

# X_tensor = torch.tensor(X, dtype=torch.float)  # Features
# y_tensor = torch.tensor(y, dtype=torch.long)  # Labels


# Stratified train-test split
train_idx, test_idx = train_test_split(
    range(len(y)), test_size=0.2, stratify=y, random_state=42
)

# Create train and test masks
train_mask = torch.zeros(len(y), dtype=torch.bool)
test_mask = torch.zeros(len(y), dtype=torch.bool)

train_mask[train_idx] = True
test_mask[test_idx] = True

# y_tensor

tensor([0, 0, 0,  ..., 9, 9, 9])

In [42]:
# from sklearn.metrics import classification_report

# Adjust the PyTorch Geometric Data object
data = Data(x=X_tensor, edge_index=edge_index, y=y_tensor)
data.train_mask = train_mask
data.test_mask = test_mask

# Define the GNN model
model = GCN(input_dim=data.num_node_features, hidden_dim=128, output_dim=len(set(y)))

# Define optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()

# Training loop
for epoch in range(200):
    output = model.train()
    optimizer.zero_grad()

    # Forward pass
    out = model(data.x, data.edge_index)

    # Compute loss for training set
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

    # Evaluate on test set
    model.eval()
    with torch.no_grad():
        _, pred = out.max(dim=1)  # Predicted labels
        correct = int((pred[data.test_mask] == data.y[data.test_mask]).sum())
        test_acc = correct / int(data.test_mask.sum())
        predictions = model(data.x, data.edge_index).argmax(dim=1)
        correct = (predictions[data.test_mask] == data.y[data.test_mask]).sum()
        acc = int(correct) / int(data.test_mask.sum())

    # # Print metrics every 10 epochs
    if epoch % 10 == 0:
        print(f"Epoch {epoch:03d}, Loss: {loss.item():.4f}, Test Acc: {test_acc:.4f}")
        # test_labels = data.y[test_mask].cpu().numpy()
        # test_preds = pred[test_mask].cpu().numpy()

        # # Compute metrics
        # report = classification_report(test_labels, test_preds, output_dict=True)

    # if epoch % 10 == 0:
    #     print(f"Epoch {epoch:03d}, Loss: {loss.item():.4f}, Test Acc: {test_acc:.4f}, "
    #           f"Precision: {test_precision:.4f}, Recall: {test_recall:.4f}")


Epoch 000, Loss: 2.6010, Test Acc: 0.1326
Epoch 010, Loss: 1.6125, Test Acc: 0.4274
Epoch 020, Loss: 1.3187, Test Acc: 0.5325
Epoch 030, Loss: 1.1697, Test Acc: 0.5981
Epoch 040, Loss: 1.0457, Test Acc: 0.6471
Epoch 050, Loss: 0.9713, Test Acc: 0.6682
Epoch 060, Loss: 0.9014, Test Acc: 0.6867
Epoch 070, Loss: 0.8519, Test Acc: 0.7047
Epoch 080, Loss: 0.8032, Test Acc: 0.7202
Epoch 090, Loss: 0.7626, Test Acc: 0.7262
Epoch 100, Loss: 0.7271, Test Acc: 0.7397
Epoch 110, Loss: 0.6949, Test Acc: 0.7593
Epoch 120, Loss: 0.6684, Test Acc: 0.7578
Epoch 130, Loss: 0.6450, Test Acc: 0.7663
Epoch 140, Loss: 0.6221, Test Acc: 0.7768
Epoch 150, Loss: 0.6098, Test Acc: 0.7748
Epoch 160, Loss: 0.5820, Test Acc: 0.7903
Epoch 170, Loss: 0.5661, Test Acc: 0.7938
Epoch 180, Loss: 0.5497, Test Acc: 0.8003
Epoch 190, Loss: 0.5379, Test Acc: 0.8033
